# Further Pre-training MobileBERT MLM with Federated Averaging (Shakepeare)

In [1]:
# Copyright 2020, The TensorFlow Federated Authors.
# Copyright 2020, Ronald Seoh
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Google Colab settings

In [2]:
# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    # Mount Google Drive root directory
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/Colab Notebooks/BERTerated'
    
    # List the directory contents
    !ls

### CUDA Multi GPU

In [3]:
# Use this code snippet to use specific GPUs
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

# os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"

In [4]:
# IPython reloading magic
%load_ext autoreload
%autoreload 2

In [5]:
# Install required packages
# !pip install -r requirements.txt

## Import packages

In [6]:
import os
import sys
import random
import datetime
import json
import pathlib
import itertools

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_text as tf_text
import tensorflow_addons as tfa
import transformers

import nest_asyncio
nest_asyncio.apply()

import fedavg
import fedavg_client
import datasets
import utils


# Random seed settings
random_seed = 692
random.seed(random_seed) # Python
np.random.seed(random_seed) # NumPy
tf.random.set_seed(random_seed) # TensorFlow

# Test if TFF is working
tff.federated_computation(lambda: 'Hello, World!')()

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:43: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20201126). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


b'Hello, World!'

In [7]:
# Print version information
print("Python version: " + sys.version)
print("NumPy version: " + np.__version__)
print("TensorFlow version: " + tf.__version__)
print("TensorFlow Federated version: " + tff.__version__)
print("Transformers version: " + transformers.__version__)

Python version: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
NumPy version: 1.18.4
TensorFlow version: 2.5.0-dev20201126
TensorFlow Federated version: 0.17.0
Transformers version: 3.4.0


In [8]:
!nvidia-smi

Fri Nov 27 06:19:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 450.57       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:81:00.0 Off |                  N/A |
|  0%   20C    P2    54W / 250W |  10377MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:82:00.0 Off |                  N/A |
|  0%   

In [9]:
tf_logical_devices_cpu = tf.config.list_logical_devices('CPU')
tf_logical_devices_gpu = tf.config.list_logical_devices('GPU')

## Experiment Settings

In [10]:
EXPERIMENT_CONFIG = {}

EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'] = 'google/mobilebert-uncased'
EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'] = os.path.join('.', 'transformers_cache')

EXPERIMENT_CONFIG['TOTAL_ROUNDS'] = 1000 # Number of total training rounds
EXPERIMENT_CONFIG['ROUNDS_PER_EVAL'] = 1 # How often to evaluate

EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'] = 10 # How many clients to sample per round.
EXPERIMENT_CONFIG['CLIENT_EPOCHS_PER_ROUND'] = 3

EXPERIMENT_CONFIG['BATCH_SIZE'] = 8 # Batch size used on the client.
EXPERIMENT_CONFIG['TEST_BATCH_SIZE'] = 16 # Minibatch size of test data.

# Maximum length of input token sequence for BERT.
EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'] = 128

# Optimizer configuration
EXPERIMENT_CONFIG['SERVER_LEARNING_RATE'] = 1.0 # Server learning rate.
EXPERIMENT_CONFIG['CLIENT_LEARNING_RATE'] = 3e-5 # Client learning rate

# Client dataset setting
EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT'] = -1
EXPERIMENT_CONFIG['TEST_NUM_CLIENT_LIMIT'] = -1

# Path to save trained weights and logs
EXPERIMENT_CONFIG['RESULTS_DIRECTORY'] = os.path.join(
    '.', 'results',
    'mobilebert_mlm_shakespeare_fedavg',
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
)

EXPERIMENT_CONFIG['RESULTS_LOG'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "logs")
EXPERIMENT_CONFIG['RESULTS_MODEL'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "model")
EXPERIMENT_CONFIG['RESULTS_CONFIG'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "config")

In [11]:
# Dump all the configuration into a json file
pathlib.Path(EXPERIMENT_CONFIG['RESULTS_CONFIG']).mkdir(parents=True, exist_ok=True)

with open(os.path.join(EXPERIMENT_CONFIG['RESULTS_CONFIG'], "config.json"), 'w') as config_file:
    json.dump(EXPERIMENT_CONFIG, config_file, indent=6)

In [12]:
# TFF executor factory settings
# Reference: https://www.tensorflow.org/federated/api_docs/python/tff/framework/local_executor_factory
tff.backends.native.set_local_execution_context(
    num_clients=EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'],
    max_fanout=100,
    clients_per_thread=1,
    server_tf_device=tf_logical_devices_cpu[0],
    client_tf_devices=tf_logical_devices_gpu,
)

## Dataset

### Dataset loader

In [13]:
train_client_data, test_client_data = tff.simulation.datasets.shakespeare.load_data(cache_dir='./tff_cache')

### Tokenizer

In [14]:
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(
    EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'], cache_dir=EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'])

In [15]:
# Imitate transformers tokenizer with TF.Text Tokenizer
tokenizer_tf_text, vocab_lookup_table, special_ids_mask_table = \
datasets.preprocessing_for_bert.convert_huggingface_tokenizer(bert_tokenizer)

### Preprocessing

In [16]:
def check_empty_snippet(x):
    return tf.strings.length(x['snippets']) > 0

def tokenizer_and_mask_wrapped(x):

    masked, labels = datasets.preprocessing_for_bert.tokenize_and_mask(tf.reshape(x['snippets'], shape=[1]),
                                                                       max_seq_length=EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'],
                                                                       bert_tokenizer_tf_text=tokenizer_tf_text,
                                                                       vocab_lookup_table=vocab_lookup_table,
                                                                       special_ids_mask_table=special_ids_mask_table,
                                                                       cls_token_id=bert_tokenizer.cls_token_id,
                                                                       sep_token_id=bert_tokenizer.sep_token_id,
                                                                       pad_token_id=bert_tokenizer.pad_token_id,
                                                                       mask_token_id=bert_tokenizer.mask_token_id)

    return (masked, labels)

def preprocess_for_train(train_dataset):
    return (
        train_dataset
        # Tokenize each samples using MobileBERT tokenizer
        #.map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=20, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly BATCH_SIZE
        # and make the shape **exactly** (BATCH_SIZE, SEQ_LENGTH)
        .batch(EXPERIMENT_CONFIG['BATCH_SIZE'])#, drop_remainder=True)
        # Repeat to make each client train multiple epochs
        .repeat(count=EXPERIMENT_CONFIG['CLIENT_EPOCHS_PER_ROUND'])
    )
    
def preprocess_for_test(test_dataset):
    return (
        test_dataset
        # Tokenize each samples using MobileBERT tokenizer
        #.map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=20, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly TEST_BATCH_SIZE
        # and make the shape **exactly** (TEST_BATCH_SIZE, SEQ_LENGTH)
        .batch(EXPERIMENT_CONFIG['TEST_BATCH_SIZE'])#, drop_remainder=True)
    )   

### Training set

In [17]:
# Since the dataset is pretty large, we randomly select TRAIN_NUM_CLIENT_LIMIT number of clients.
all_train_client_ids = train_client_data.client_ids

random.shuffle(all_train_client_ids)

if EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT'] > 0:
    selected_train_client_ids = all_train_client_ids[0:EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT']]
else:
    selected_train_client_ids = all_train_client_ids

In [18]:
train_client_data = train_client_data.preprocess(preprocess_fn=lambda x: x.filter(check_empty_snippet))

In [19]:
train_client_data = train_client_data.preprocess(preprocess_fn=preprocess_for_train)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [20]:
print(train_client_data.element_type_structure)

(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None))


In [21]:
train_client_states = {}

# Initialize client states for all clients (selected for the entire simulation)
for i, client_id in enumerate(selected_train_client_ids):
    train_client_states[client_id] = fedavg_client.ClientState(client_serial=i, visit_count=0)

### Test set

In [22]:
test_client_data_all_merged = test_client_data.create_tf_dataset_for_client(
    test_client_data.client_ids[0]).filter(check_empty_snippet)

if len(test_client_data.client_ids) > 1:
    for i in range(1, len(test_client_data.client_ids)):
        test_client_data_all_merged = test_client_data_all_merged.concatenate(
            test_client_data.create_tf_dataset_for_client(test_client_data.client_ids[i]).filter(check_empty_snippet))

In [23]:
test_client_data_all_merged = preprocess_for_test(test_client_data_all_merged)

In [24]:
test_client_data_all_merged = test_client_data_all_merged.cache()

In [25]:
test_client_data_all_merged = test_client_data_all_merged.prefetch(10000)

In [26]:
print(test_client_data_all_merged.element_spec)

(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None))


## Model

In [27]:
bert_model = transformers.TFAutoModelForPreTraining.from_pretrained(
    EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'], cache_dir=EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'])

All model checkpoint layers were used when initializing TFMobileBertForPreTraining.

All the layers of TFMobileBertForPreTraining were initialized from the model checkpoint at google/mobilebert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMobileBertForPreTraining for predictions without further training.


In [28]:
print(bert_model.config)

MobileBertConfig {
  "_name_or_path": "google/mobilebert-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_activation": false,
  "embedding_size": 128,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "intra_bottleneck_size": 128,
  "key_query_shared_bottleneck": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "mobilebert",
  "normalization_type": "no_norm",
  "num_attention_heads": 4,
  "num_feedforward_networks": 4,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "trigram_input": true,
  "true_hidden_size": 128,
  "type_vocab_size": 2,
  "use_bottleneck": true,
  "use_bottleneck_attention": false,
  "vocab_size": 30522
}



In [29]:
# Due to the limitations with Keras subclasses, we can only use the main layer part from pretrained models
# and add output heads by ourselves
bert_keras_converted = utils.convert_huggingface_mlm_to_keras(
    huggingface_model=bert_model,
    max_seq_length=EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'],
)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [30]:
# Use lists of NumPy arrays to backup pretained weights
bert_pretrained_trainable_weights = []
bert_pretrained_non_trainable_weights = []

for w in bert_keras_converted.trainable_weights:
    bert_pretrained_trainable_weights.append(w.numpy())

for w in bert_keras_converted.non_trainable_weights:
    bert_pretrained_non_trainable_weights.append(w.numpy())

In [31]:
def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""

    loss = utils.MaskedLMCrossEntropy()

    model_wrapped = utils.KerasModelWrapper(
        tf.keras.models.clone_model(bert_keras_converted),
        train_client_data.element_type_structure, loss)

    return model_wrapped

## Training

### Training setups

In [32]:
summary_writer = tf.summary.create_file_writer(EXPERIMENT_CONFIG['RESULTS_LOG'])

In [33]:
def server_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=EXPERIMENT_CONFIG['SERVER_LEARNING_RATE'])

def client_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=EXPERIMENT_CONFIG['CLIENT_LEARNING_RATE'])

In [34]:
iterative_process = fedavg.build_federated_averaging_process(
    model_fn=tff_model_fn,
    model_input_spec=train_client_data.element_type_structure,
    initial_trainable_weights=bert_pretrained_trainable_weights,
    initial_non_trainable_weights=bert_pretrained_non_trainable_weights,
    server_optimizer_fn=server_optimizer_fn, 
    client_optimizer_fn=client_optimizer_fn)

In [35]:
server_state = iterative_process.initialize()

In [36]:
metric_eval = tfa.metrics.MeanMetricWrapper(fn=utils.calculate_masked_lm_cross_entropy, name='test_ce')

In [37]:
model_final = tff_model_fn() # The one to store the final weights

### Training loop

In [ ]:
#sys.stdout = open(os.path.join(EXPERIMENT_CONFIG['RESULTS_LOG'], 'training_loop_output.txt'), 'w')

with summary_writer.as_default():
    for round_num in range(1, EXPERIMENT_CONFIG['TOTAL_ROUNDS'] + 1):        
        # FedAvg
        print(f'Round {round_num} start!')

        # Training client selection
        sampled_client_serials = np.random.choice(
            len(selected_train_client_ids),
            size=EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'],
            replace=False)

        sampled_train_data = [
            train_client_data.create_tf_dataset_for_client(selected_train_client_ids[client_serial])
            for client_serial in sampled_client_serials
        ]
        
        sampled_client_states = [
            train_client_states[selected_train_client_ids[client_serial]]
            for client_serial in sampled_client_serials
        ]
        
        print("Selected client serials:", sampled_client_serials)

        server_state, new_client_states, train_loss = iterative_process.next(
            server_state, sampled_client_states, sampled_train_data)

        print(f'Round {round_num} training loss: {train_loss}')
        
        # Record the current round's training loss to the log
        tf.summary.scalar('train_loss', train_loss, step=round_num)
        
        print()
        
        # Update client states
        print("Updating client states.")

        for state in new_client_states:
            train_client_states[selected_train_client_ids[state.client_serial]] = state

        print()

        # Evaluation
        if round_num % EXPERIMENT_CONFIG['ROUNDS_PER_EVAL'] == 0:
            model_final.from_weights(server_state.model_weights)

            # Test dataset generation for this round
            print("Calculating validation metric:")

            validation_metric = utils.keras_evaluate(
                model_final.keras_model, test_client_data_all_merged, metric_eval)

            print(f'Round {round_num} validation metric: {validation_metric}')
            
            # Write down train_metrics to the log
            tf.summary.scalar('validation_metric', validation_metric, step=round_num)
            
        print()
        
#sys.stdout.close()

Round 1 start!
Selected client serials: [267 191 488 146  82 205 384  38  86  78]
Anonymous client 78 : updated the model with server message.
Anonymous client 78 : training start.
Anonymous client 78 : batch 1 , 8 examples processed
Anonymous client 78 : batch 2 , 16 examples processed
Anonymous client 78 : batch 3 , 24 examples processed
Anonymous client 78 : batch 4 , 32 examples processed
Anonymous client 78 : batch 5 , 40 examples processed
Anonymous client 78 : batch 6 , 48 examples processed
Anonymous client 78 : batch 7 , 56 examples processed
Anonymous client 78 : batch 8 , 60 examples processed
Anonymous client 78 : batch 9 , 68 examples processed
Anonymous client 78 : batch 10 , 76 examples processed
Anonymous client 78 : batch 11 , 84 examples processed
Anonymous client 78 : batch 12 , 92 examples processed
Anonymous client 78 : batch 13 , 100 examples processed
Anonymous client 78 : batch 14 , 108 examples processed
Anonymous client 78 : batch 15 , 116 examples processed
A

Anonymous client 295 : updated the model with server message.
Anonymous client 295 : training start.
Anonymous client 601 : updated the model with server message.
Anonymous client 601 : training start.
Anonymous client 714 : batch 1 , 8 examples processed
Anonymous client 547 : updated the model with server message.
Anonymous client 547 : training start.
Anonymous client 660 : updated the model with server message.
Anonymous client 660 : training start.
Anonymous client 232 : updated the model with server message.
Anonymous client 232 : training start.
Anonymous client 647 : updated the model with server message.
Anonymous client 647 : training start.
Anonymous client 629 : updated the model with server message.
Anonymous client 629 : training start.
Anonymous client 292 : batch 1 , 1 examples processed
Anonymous client 295 : batch 1 , 8 examples processed
Anonymous client 629 : batch 1 , 5 examples processed
Anonymous client 232 : batch 1 , 1 examples processed
Anonymous client 292 : 

Anonymous client 547 : batch 22 , 173 examples processed
Anonymous client 660 : batch 23 , 184 examples processed
Anonymous client 161 : batch 40 , 314 examples processed
Anonymous client 660 : batch 24 , 189 examples processed
Anonymous client 547 : batch 23 , 181 examples processed
Anonymous client 161 : batch 41 , 322 examples processed
Anonymous client 161 : batch 42 , 327 examples processed
Anonymous client 547 : batch 24 , 189 examples processed
Anonymous client 660 : batch 25 , 197 examples processed
Anonymous client 161 : training finished. 327  examples processed, loss: 10.322876
Anonymous client 547 : batch 25 , 197 examples processed
Anonymous client 660 : batch 26 , 205 examples processed
Anonymous client 547 : batch 26 , 205 examples processed
Anonymous client 660 : batch 27 , 213 examples processed
Anonymous client 547 : batch 27 , 213 examples processed
Anonymous client 660 : batch 28 , 221 examples processed
Anonymous client 547 : batch 28 , 221 examples processed
Anony

Anonymous client 521 : training finished. 3  examples processed, loss: 10.323535
Anonymous client 389 : batch 4 , 32 examples processed
Anonymous client 185 : training finished. 15  examples processed, loss: 10.320982
Anonymous client 171 : batch 4 , 30 examples processed
Anonymous client 218 : batch 5 , 34 examples processed
Anonymous client 415 : batch 5 , 33 examples processed
Anonymous client 389 : batch 5 , 40 examples processed
Anonymous client 412 : batch 5 , 40 examples processed
Anonymous client 171 : batch 5 , 38 examples processed
Anonymous client 570 : batch 5 , 38 examples processed
Anonymous client 389 : batch 6 , 48 examples processed
Anonymous client 218 : batch 6 , 39 examples processed
Anonymous client 171 : batch 6 , 44 examples processed
Anonymous client 415 : batch 6 , 41 examples processed
Anonymous client 412 : batch 6 , 48 examples processed
Anonymous client 389 : batch 7 , 56 examples processed
Anonymous client 570 : batch 6 , 45 examples processed
Anonymous cl

Anonymous client 23 : batch 6 , 27 examples processed
Anonymous client 41 : batch 9 , 72 examples processed
Anonymous client 243 : batch 7 , 56 examples processed
Anonymous client 382 : batch 7 , 44 examples processed
Anonymous client 41 : batch 10 , 74 examples processed
Anonymous client 23 : training finished. 27  examples processed, loss: 9.93844
Anonymous client 243 : batch 8 , 64 examples processed
Anonymous client 41 : batch 11 , 82 examples processed
Anonymous client 382 : batch 8 , 52 examples processed
Anonymous client 41 : batch 12 , 90 examples processed
Anonymous client 243 : batch 9 , 72 examples processed
Anonymous client 41 : batch 13 , 98 examples processed
Anonymous client 382 : batch 9 , 54 examples processed
Anonymous client 243 : batch 10 , 80 examples processed
Anonymous client 41 : batch 14 , 106 examples processed
Anonymous client 382 : training finished. 54  examples processed, loss: 10.319067
Anonymous client 41 : batch 15 , 114 examples processed
Anonymous cli

...

Anonymous client 392 : batch 18 , 138 examples processed
Anonymous client 56 : batch 17 , 132 examples processed
Anonymous client 392 : batch 19 , 146 examples processed
Anonymous client 56 : batch 18 , 140 examples processed
Anonymous client 392 : batch 20 , 154 examples processed
Anonymous client 56 : batch 19 , 148 examples processed
Anonymous client 392 : batch 21 , 162 examples processed
Anonymous client 56 : batch 20 , 156 examples processed
Anonymous client 392 : batch 22 , 170 examples processed
Anonymous client 56 : batch 21 , 164 examples processed
Anonymous client 392 : batch 23 , 178 examples processed
Anonymous client 56 : batch 22 , 172 examples processed
Anonymous client 392 : batch 24 , 186 examples processed
Anonymous client 56 : batch 23 , 180 examples processed
Anonymous client 392 : batch 25 , 194 examples processed
Anonymous client 56 : batch 24 , 188 examples processed
Anonymous client 392 : batch 26 , 202 examples processed
Anonymous client 56 : batch 25 , 196 ex

Anonymous client 684 : batch 9 , 66 examples processed
Anonymous client 154 : batch 9 , 54 examples processed
Anonymous client 563 : batch 9 , 72 examples processed
Anonymous client 527 : training finished. 72  examples processed, loss: 10.1926146
Anonymous client 684 : batch 10 , 74 examples processed
Anonymous client 154 : training finished. 54  examples processed, loss: 10.1911821
Anonymous client 563 : batch 10 , 80 examples processed
Anonymous client 684 : batch 11 , 82 examples processed
Anonymous client 563 : batch 11 , 88 examples processed
Anonymous client 684 : batch 12 , 87 examples processed
Anonymous client 563 : batch 12 , 96 examples processed
Anonymous client 684 : training finished. 87  examples processed, loss: 10.2065916
Anonymous client 563 : batch 13 , 97 examples processed
Anonymous client 563 : batch 14 , 105 examples processed
Anonymous client 563 : batch 15 , 113 examples processed
Anonymous client 563 : batch 16 , 121 examples processed
Anonymous client 563 : 

Anonymous client 152 : batch 16 , 121 examples processed
Anonymous client 290 : training finished. 132  examples processed, loss: 10.2042751
Anonymous client 152 : batch 17 , 129 examples processed
Anonymous client 152 : batch 18 , 137 examples processed
Anonymous client 152 : batch 19 , 145 examples processed
Anonymous client 152 : batch 20 , 146 examples processed
Anonymous client 152 : batch 21 , 154 examples processed
Anonymous client 152 : batch 22 , 162 examples processed
Anonymous client 152 : batch 23 , 170 examples processed
Anonymous client 152 : batch 24 , 178 examples processed
Anonymous client 152 : batch 25 , 186 examples processed
Anonymous client 152 : batch 26 , 194 examples processed
Anonymous client 152 : batch 27 , 202 examples processed
Anonymous client 152 : batch 28 , 210 examples processed
Anonymous client 152 : batch 29 , 218 examples processed
Anonymous client 152 : batch 30 , 219 examples processed
Anonymous client 152 : training finished. 219  examples proce

Round 60 training loss: 10.191659927368164

Updating client states.

Calculating validation metric:
Round 60 validation metric: 10.19632625579834

Round 61 start!
Selected client serials: [562 110 535 664 172 134 469  90  21  38]
Anonymous client 562 : updated the model with server message.
Anonymous client 562 : training start.
Anonymous client 664 : updated the model with server message.
Anonymous client 664 : training start.
Anonymous client 134 : updated the model with server message.
Anonymous client 134 : training start.
Anonymous client 535 : updated the model with server message.
Anonymous client 535 : training start.
Anonymous client 21 : updated the model with server message.
Anonymous client 21 : training start.
Anonymous client 469 : updated the model with server message.
Anonymous client 469 : training start.
Anonymous client 90 : updated the model with server message.
Anonymous client 90 : training start.
Anonymous client 110 : updated the model with server message.
Anony

Anonymous client 216 : batch 3 , 24 examples processed
Anonymous client 641 : batch 3 , 17 examples processed
Anonymous client 362 : batch 3 , 20 examples processed
Anonymous client 638 : batch 3 , 6 examples processed
Anonymous client 685 : batch 3 , 15 examples processed
Anonymous client 670 : batch 3 , 12 examples processed
Anonymous client 369 : batch 3 , 21 examples processed
Anonymous client 181 : batch 3 , 9 examples processed
Anonymous client 659 : batch 3 , 6 examples processed
Anonymous client 216 : batch 4 , 32 examples processed
Anonymous client 397 : training finished. 18  examples processed, loss: 10.2217169
Anonymous client 641 : batch 4 , 18 examples processed
Anonymous client 362 : batch 4 , 28 examples processed
Anonymous client 638 : training finished. 6  examples processed, loss: 10.2125921
Anonymous client 369 : batch 4 , 29 examples processed
Anonymous client 670 : training finished. 12  examples processed, loss: 10.1342621
Anonymous client 685 : training finished

### Save the trained model

In [ ]:
model_final.keras_model.save(EXPERIMENT_CONFIG['RESULTS_MODEL'])

## Evaluation

In [ ]:
test_metric = utils.keras_evaluate(
    model_final.keras_model, test_client_data_all_merged, metric_eval)

print(test_metric)

In [ ]:
# Debug: Did the client states get updated correctly?
print(train_client_states)